Адаптируем построение бандла из задачи поиска ошибки согласования рода к задаче поиска антецедентов.

Посмотрим как работает CoreExtractor - сущность, которая по слову вернет все признаки. Умеет работать с любыми индексами.

In [1]:
from sklearn import datasets
import pandas as pd
from yo_fluq_ds import *
from tg.common.ml import batched_training as bt
from tg.common import DataBundle, Loc
from tg.common.ml.batched_training import IndexedDataBundle
from tg.grammar_ru.ml.components.core_extractor.extractor import CoreExtractor
from tg.grammar_ru.ml.components.plain_context_builder import PlainContextBuilder
from tg.grammar_ru.ml.components.contextual_binding import ContextualBinding, ContextualNetworkType
from tg.common.ml.batched_training import mirrors as btm
from tg.common.ml.batched_training import Batcher
from tg.grammar_ru.common import Separator

In [2]:
from tg.grammar_ru.common import Loc
from tg.grammar_ru.ml.features import PyMorphyFeaturizer

db = DataBundle.load(Loc.bundles_path/'antcd/simple') #TODO word_distance
db['index']

,pronoun_word_id,pronoun_sentence_id,candidate_word_id,candidate_sentence_id,candidate_distance,is_match
sample_id,,,,,,
0,7,0,4,0,1,1.0
1,7,0,5,0,0,0.0
2,514887,514880,514884,514880,1,1.0
3,514887,514880,514885,514880,0,0.0
4,1903357,1903337,1903342,1903335,3,0.0
...,...,...,...,...,...,...
101060,22837552,22837430,22837513,22837429,4,0.0
101061,22837552,22837430,22837524,22837430,3,0.0
101062,22837552,22837430,22837537,22837430,2,0.0


In [3]:
db.index = db.index.iloc[:1000]
idb = IndexedDataBundle(db.index, db)
idb

In [4]:
Separator.Viewer().tooltip("word_id").to_html_display(
    idb.bundle.src.loc[idb.bundle.src.sentence_id == 0])


CoreExtractor принимает список extractor'ов и применяет их к бандлу. Потом нужно подать его ContextualBinding. TODO да?

##### Пример CoreExtractor

In [5]:
core = CoreExtractor(allow_list=['slovnet'], join_column='pronoun_word_id')  # TODO что делает allowlist
core.fit(idb)


2022-11-25 12:44:33.844897+00:00 INFO: Fitting extractor pymorphy in CoreExtractor
2022-11-25 12:44:34.088402+00:00 INFO: Success
2022-11-25 12:44:34.088402+00:00 INFO: Fitting extractor slovnet_morph in CoreExtractor
2022-11-25 12:44:34.315028+00:00 INFO: Success
2022-11-25 12:44:34.315028+00:00 INFO: Fitting extractor slovnet_syntax in CoreExtractor
2022-11-25 12:44:34.498407+00:00 INFO: Success
2022-11-25 12:44:34.498407+00:00 INFO: Fitting extractor syntax_fixes in CoreExtractor
2022-11-25 12:44:34.498407+00:00 INFO: Skipped as the corresponding frame is missing from the bundle
2022-11-25 12:44:34.498407+00:00 INFO: Fitting extractor syntax_stats in CoreExtractor
2022-11-25 12:44:34.498407+00:00 INFO: Skipped as the corresponding frame is missing from the bundle


In [6]:
core.extractors.keys()
[(e_name, e.disabled) for e_name, e in core.extractors.items()]

[('pymorphy', False),
 ('slovnet_morph', False),
 ('slovnet_syntax', False),
 ('syntax_fixes', True),
 ('syntax_stats', True)]

In [7]:
extracted = core.extract(idb)
extracted

,pymorphy_score,pymorphy_delta_score,pymorphy_alternatives,pymorphy_POS_NPRO,pymorphy_animacy_NULL,pymorphy_gender_masc,pymorphy_gender_femn,pymorphy_number_sing,pymorphy_case_accs,pymorphy_case_nomn,...,slovnet_morph_Voice_NULL,slovnet_morph_Degree_NULL,slovnet_morph_Foreign_NULL,slovnet_morph_Variant_NULL,slovnet_morph_Polarity_NULL,slovnet_syntax_relation_nmod,slovnet_syntax_relation_det,slovnet_syntax_relation_obj,slovnet_syntax_relation_nsubj,slovnet_syntax_relation_iobj
sample_id,,,,,,,,,,,,,,,,,,,,,
0,0.039619,0.053981,0.247520,1.0,1.0,1.0,0.0,1.0,1.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
1,0.039619,0.053981,0.247520,1.0,1.0,1.0,0.0,1.0,1.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
2,0.039619,0.053981,0.247520,1.0,1.0,1.0,0.0,1.0,1.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
3,0.039619,0.053981,0.247520,1.0,1.0,1.0,0.0,1.0,1.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
4,-0.908753,-0.877471,0.097863,1.0,1.0,0.0,1.0,1.0,1.0,0.0,...,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1.164046,0.228254,-3.433975,1.0,1.0,0.0,1.0,1.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
996,1.164046,0.228254,-3.433975,1.0,1.0,0.0,1.0,1.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
997,1.164046,0.228254,-3.433975,1.0,1.0,0.0,1.0,1.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0


* Для каждого слова получили все признаки из pymorphy, slovnet...

* Нам необходим контекст слова

Будем использовать `PlainContextBuilder`.  Он построит двойной индекс.

* хотим исключить само слово из контекста, поэтому `include_zero_offset=False`

#### Пример PlainContextBuilder

In [8]:
pcb = PlainContextBuilder(include_zero_offset=True,
                          left_to_right_contexts_proportion=0.5)
pcb.sentence_id_column_name = 'pronoun_sentence_id'
pcb.word_id_column_name = 'pronoun_word_id'
contexts = pcb.build_context(idb, context_size=5)
contexts  # TODO у некоторых только offsset == 1


another_word_id
sample_id offset                 
0         0                     7
1         0                     7
2         0                514887
3         0                514887
4         0               1903357
...                           ...
997       2               2011235
998       1               2021284
          2               2021285
999       1               2021284
          2               2021285

[4027 rows x 1 columns]

In [9]:
cdf = contexts.reset_index()
cdf.loc[cdf.sample_id == 2]

,sample_id,offset,another_word_id
2,2,0,514887
1004,2,-1,514886
1005,2,-2,514885
2031,2,1,514888
2032,2,2,514889


#### WordContextAssemblyPoint (~ ContextualBinding)

Задача AssemblyPoint'a (он же ContextualBinding) - порождать экстракторами батчи в том виде, в котором они будут приняты сетью. Для LSTM будет 3d-тензор. Для Plain-сети будет плоский датафрейм.

In [10]:
plain_context = ContextualBinding(
    name='plain_context',
    context_length=3,
    network_type=ContextualNetworkType.Plain,
    hidden_size=[30],
    context_builder=pcb,
    extractor=CoreExtractor(
        allow_list=['slovnet'], join_column='another_word_id'),
    debug=False
)


**Пояснение:**
По умолчанию CoreExtractor пытается мерджить по word_id. В нашем случае CoreExtractor отработает после создания PlainContextBuilder'ом двойного индекса. 
Для добавления признаков слов из контекста будем join'ить по столбцу another_word_id. Поэтому join_column='another_word_id'

Для создания согласованных экстракторов и сетей у AssemblyPoint есть методы create_extractor и create_network.

У такой сети на последнем слое hidden_size нейронов. Ее выход можно подать в другую сеть, которая будет выдавать вероятности классов.

In [11]:
context_extractor = plain_context.create_extractor(task=None, bundle=idb)
context_extractor.fit(idb)
# not_batch = core_extractor.extract(idb)


2022-11-25 12:44:35.461137+00:00 INFO: Fitting extractor pymorphy in CoreExtractor
2022-11-25 12:44:35.693815+00:00 INFO: Success
2022-11-25 12:44:35.693815+00:00 INFO: Fitting extractor slovnet_morph in CoreExtractor
2022-11-25 12:44:35.969500+00:00 INFO: Success
2022-11-25 12:44:35.970494+00:00 INFO: Fitting extractor slovnet_syntax in CoreExtractor
2022-11-25 12:44:36.263570+00:00 INFO: Success
2022-11-25 12:44:36.264570+00:00 INFO: Fitting extractor syntax_fixes in CoreExtractor
2022-11-25 12:44:36.266569+00:00 INFO: Skipped as the corresponding frame is missing from the bundle
2022-11-25 12:44:36.274573+00:00 INFO: Fitting extractor syntax_stats in CoreExtractor
2022-11-25 12:44:36.277572+00:00 INFO: Skipped as the corresponding frame is missing from the bundle


In [12]:
factory = plain_context.create_network_factory(
    task=None, input=None)  # None это ок. это legacy


#### Batcher

Иногда нам нужны фичи контекстов нескольких слов. Например, в задаче поиска антецедентов это местоимение и два существительных - потенциальные антецеденты. Для каждого из этих слов создадим экстрактор (для каждого экстрактора будет отдельная голова нейросети). Batcher примет список этих экстракторов и создаст батч.

`+` нам понадобится экстрагировать лейблы

In [13]:
from tg.grammar_ru.ml.components.training_task_factory import Conventions
from tg.common.ml import dft
# экстрагирует лейблы. Получается one-hot df. Лейблы пойдут в loss function.


def get_binary_extractor():
    label_extractor = (bt.PlainExtractor
                       .build(Conventions.LabelFrame)
                       .index()
                       .apply(take_columns=['is_match'], transformer=None)
                       )
    return label_extractor


def test_extractor(extractor, bundle):
    extractor.fit(bundle)
    return extractor.extract(bundle)

test_extractor(get_binary_extractor(), idb).head(10)


,is_match
sample_id,
0,1.0
1,0.0
2,1.0
3,0.0
4,0.0
5,0.0
6,1.0
7,0.0
8,0.0


In [14]:
batcher = Batcher(batch_size=1000, extractors=[
                  context_extractor, get_binary_extractor()])
batch = batcher.fit_extract(idb)


2022-11-25 12:44:38.434987+00:00 INFO: Fitting extractor pymorphy in CoreExtractor
2022-11-25 12:44:38.883097+00:00 INFO: Success
2022-11-25 12:44:38.884097+00:00 INFO: Fitting extractor slovnet_morph in CoreExtractor
2022-11-25 12:44:39.137812+00:00 INFO: Success
2022-11-25 12:44:39.153439+00:00 INFO: Fitting extractor slovnet_syntax in CoreExtractor
2022-11-25 12:44:39.414864+00:00 INFO: Success
2022-11-25 12:44:39.418864+00:00 INFO: Fitting extractor syntax_fixes in CoreExtractor
2022-11-25 12:44:39.434871+00:00 INFO: Skipped as the corresponding frame is missing from the bundle
2022-11-25 12:44:39.440869+00:00 INFO: Fitting extractor syntax_stats in CoreExtractor
2022-11-25 12:44:39.445889+00:00 INFO: Skipped as the corresponding frame is missing from the bundle


In [15]:
list(batch)
batch['plain_context']

,f0a0_offset_is_presenting_at_-1,f0a0_offset_is_presenting_at_0,f0a0_offset_is_presenting_at_1,f0a0_pymorphy_POS_ADJF_at_-1,f0a0_pymorphy_POS_ADJF_at_0,f0a0_pymorphy_POS_ADJF_at_1,f0a0_pymorphy_POS_CONJ_at_-1,f0a0_pymorphy_POS_CONJ_at_0,f0a0_pymorphy_POS_CONJ_at_1,f0a0_pymorphy_POS_NONE_at_-1,...,f0a0_slovnet_syntax_relation_parataxis_at_-1,f0a0_slovnet_syntax_relation_parataxis_at_0,f0a0_slovnet_syntax_relation_parataxis_at_1,f0a0_slovnet_syntax_relation_punct_at_-1,f0a0_slovnet_syntax_relation_punct_at_0,f0a0_slovnet_syntax_relation_punct_at_1,f0a0_slovnet_syntax_relation_root_at_-1,f0a0_slovnet_syntax_relation_root_at_0,f0a0_slovnet_syntax_relation_root_at_1,f0a0_present_f0a0
sample_id,,,,,,,,,,,,,,,,,,,,,
0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
996,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
997,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [16]:
network = factory.create_network(task=None, input=batch)
# при создании сети необходим батч, чтобы TODO
network(batch).shape


torch.Size([1000, 30])

In [17]:
# TODO tsa run_tarining. factory...


In [18]:
# batch['label']


Полные данные не помещаются в память локальной машины, но для инициализации сети нужно понять, сколько бывает классов в полных данных. Поэтому сеть инициализируется в BatchedTrainingTask, который выполняется на сервере. 

Создадим TaskFactory, отправим ее на сервер. Она подтянет данные, создаст BatchedTrainingTask и запустит его.



In [19]:
import torch


def _update_sizes_with_argument(argument_name, argument, sizes, modificator):
    if argument is None:
        return sizes
    elif isinstance(argument, torch.Tensor):
        return modificator(sizes, argument.shape[1])
    elif isinstance(argument, pd.DataFrame):
        return modificator(sizes, argument.shape[1])
    elif isinstance(argument, int):
        return modificator(sizes, argument)
    else:
        raise ValueError(
            f"Argument {argument_name} is supposed to be int, Tensor or none, but was `{argument}`")


class FullyConnectedNetwork(torch.nn.Module):
    def __init__(self,
                 sizes: List[int],
                 input: Union[None, torch.Tensor, int] = None,
                 output: Union[None, torch.Tensor, int] = None):
        super(FullyConnectedNetwork, self).__init__()
        sizes = _update_sizes_with_argument(
            'input', input, sizes, lambda s, v: [v] + s)
        sizes = _update_sizes_with_argument(
            'output', output, sizes, lambda s, v: s + [v])
        self.layers = torch.nn.ModuleList()
        for i in range(len(sizes) - 1):
            self.layers.append(torch.nn.Linear(sizes[i], sizes[i + 1]))

    def forward(self, input):
        X = input
        for layer in self.layers:
            X = layer(X)
            X = torch.sigmoid(X)
        return X


# class AssembledModel(torch.nn.Module):
#     def __init__(self, head, tail):
#         super(AssembledModel, self).__init__()
#         self.head = head
#         self.tail = tail

#     def forward(self, x):
#         x = self.head(x)
#         x = self.tail(x)
#         return x


In [20]:
from tg.common.ml.batched_training.torch.networks.extracting_network import FeedForwardNetwork

class MyNetworkFactory:
    def __init__(self, nn_head_factory):
        self.nn_head_factory = nn_head_factory

    def create_network(self, task, input):  # input is batch ~ sample
        nn_head = self.nn_head_factory.create_network(task=None, input=input)
        head_out = nn_head(input)
        hidden_size = head_out.shape[1]
        labels_count = input['label'].shape[1]
        nn_tail = FullyConnectedNetwork(
            sizes=[3], input=hidden_size, output=labels_count)
        return FeedForwardNetwork(nn_head, nn_tail, torch.nn.Softmax(dim=1))


In [21]:
assembled_network_factory = MyNetworkFactory(factory)
assembled_network = assembled_network_factory.create_network(
    task=None, input=batch)


In [22]:
mini_batch_indices = batcher.get_mini_batch_indices(mini_batch_size = 10, batch = batch)
# mini_batch = batcher.get_mini_batch(index = mini_batch_indices[0], batch = batch)
# mini_batch['index'].shape

In [23]:
# Example of target with class indices
loss = torch.nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)
output = loss(input, input)
output.backward()
# # Example of target with class probabilities
# input = torch.randn(3, 5, requires_grad=True)
# target = torch.randn(3, 5).softmax(dim=1)
# output = loss(input, target)
# output.backward()
output

tensor(-4.8701, grad_fn=<DivBackward1>)

In [24]:
def train_loop(model, loss_fn, optimizer):
    for mini_batch_num, mini_batch_index in enumerate(mini_batch_indices):
        mini_batch = batcher.get_mini_batch(index = mini_batch_indices[mini_batch_num], batch = batch)
        # Compute prediction and loss
        pred = model(mini_batch)
        loss = loss_fn(pred, torch.tensor(mini_batch['label'].values))

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # if batch % 100 == 0:
        #     loss, current = loss.item(), batch * len(X)
        #     print(f"loss: {loss:>7f}  [{current:>5d}/{num_batches:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [25]:

learning_rate = 1
optimizer = torch.optim.SGD(assembled_network.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(assembled_network, loss, optimizer)
    # test_loop(test_dataloader, model, loss_fn)
print("Done!")


Epoch 1
-------------------------------
Epoch 2
-------------------------------
Epoch 3
-------------------------------
Epoch 4
-------------------------------
Epoch 5
-------------------------------
Epoch 6
-------------------------------
Epoch 7
-------------------------------
Epoch 8
-------------------------------
Epoch 9
-------------------------------
Epoch 10
-------------------------------
Done!


In [26]:
mini_batch['label']

NameError: name 'mini_batch' is not defined

In [ ]:
out = assembled_network(batch)
out.shape


In [ ]:
# batch['label']

In [ ]:
import torch


class ClassificationNetwork(torch.nn.Module):
    def __init__(self, hidden_size, sample):
        super(ClassificationNetwork, self).__init__()
        self.hidden = torch.nn.Linear(sample['features'].shape[1], hidden_size)
        self.output = torch.nn.Linear(hidden_size, sample['label'].shape[1])

    def forward(self, input):
        X = input['features']
        X = torch.tensor(X.astype(float).values).float()
        X = self.hidden(X)
        X = torch.sigmoid(X)
        X = self.output(X)
        X = torch.sigmoid(X)
        return X


In [ ]:
from tg.grammar_ru.ml.components.training_task_factory import TaskFactory, Conventions
from tg.common.ml import dft


class MulticlassMetrics(bt.Metric):
    def __init__(self, add_accuracy=True, add_rating=False):
        self.add_accuracy = add_accuracy
        self.add_rating = add_rating

    def get_names(self):
        result = []
        if self.add_accuracy:
            result.append('accuracy')
        if self.add_rating:
            result.append('rating')
        return result

    def measure(self, df, _):
        prefix = 'true_label_'
        labels = []
        for c in df.columns:
            if c.startswith(prefix):
                labels.append(c.replace(prefix, ''))

        def ustack(df, prefix, cols, name):
            df = df[[prefix+c for c in cols]]
            df.columns = [c for c in cols]
            df = df.unstack().to_frame(name)
            return df

        predicted = ustack(df, 'predicted_label_', labels, 'predicted')
        true = ustack(df, 'true_label_', labels, 'true')
        df = predicted.merge(true, left_index=True,
                             right_index=True).reset_index()
        df.columns = ['label', 'sample', 'predicted', 'true']
        df = df.feed(fluq.add_ordering_column(
            'sample', ('predicted', False), 'predicted_rating'))

        match = (df.loc[df.predicted_rating ==
                 0].set_index('sample').true > 0.5)
        rating = df.loc[df.true > 0.5].set_index('sample').predicted_rating
        result = []
        if self.add_accuracy:
            result.append(match.mean())
        if self.add_rating:
            result.append(rating.mean())
        return result


In [ ]:
def get_multilabel_extractor():
    label_extractor = (bt.PlainExtractor
                       .build(Conventions.LabelFrame)
                       .index()
                       .apply(take_columns=['label'], transformer=dft.DataFrameTransformerFactory.default_factory())
                       )
    return label_extractor


In [ ]:
class ClassificationTask(TaskFactory):
    def create_task(self, data, env):
        metrics = bt.MetricPool().add(MulticlassMetrics())  # TODO epoch_count=20
        self.instantiate_default_task(
            epoch_count=1, batch_size=10000, mini_batch_size=None, metric_pool=metrics)
        self.setup_batcher(
            # data, [get_feature_extractor(), get_multilabel_extractor()])
            data, [core, get_multilabel_extractor()])
        self.setup_model(lambda _, sample: ClassificationNetwork(
            20, sample), learning_rate=1)


task = ClassificationTask()
idb = IndexedDataBundle(db.index, db)
result = task.run(idb)
# pd.DataFrame(result['output']['history']).set_index('iteration').plot()


setup_butcher принимает экстракторы. Передадим core? или завернем его в get_feature_extractor?